In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [2]:
Datadirectory_train="training_data/" #training dataset

In [3]:
Datadirectory_test="testing_data/" #test dataset

In [4]:
Classes=["0","1","2","3","4","5","6"]

In [5]:
img_size=224 #to match the imagenet


#### Prepare the Data Generator with data augmentation and normalization

datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    shear_range=0.2,  # Randomly shear images
    zoom_range=0.2,  # Randomly zoom in/out on images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformations
)


In [13]:
def data_generator(batch_size, dataset):
    while True:
        X_batch = []
        y_batch = []
        for img, label in dataset:
            X_batch.append(img)
            y_batch.append(label)
            
            if len(X_batch) == batch_size:
                X_batch = np.array(X_batch) / 255.0  # Normalize
                y_batch = np.array(y_batch)
                yield X_batch, y_batch
                X_batch, y_batch = [], []
        
        if X_batch:
            X_batch = np.array(X_batch) / 255.0
            y_batch = np.array(y_batch)
            yield X_batch, y_batch

#### Load the Data

In [19]:
def create_data(Datadirectory):
    Data = []
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path)
                new_array = cv2.resize(img_array, (img_size, img_size))
                Data.append([new_array, class_num])
            except Exception as e:
                pass
    return Data






In [21]:
# Create the dataset
training_data = create_data(Datadirectory_train)
testing_data = create_data(Datadirectory_test)

In [23]:
# Split the dataset
train_data, val_data = train_test_split(training_data, test_size=0.2, random_state=42)  

#### Define the CNN Model

In [25]:
model = Sequential([
    Input(shape=(img_size, img_size, 3)),  # Input layer
    Conv2D(32, (3, 3), activation='relu'),#Extracts features from the input image by applying 32 filters of size 3x3.
    MaxPooling2D(2, 2),#reduces each spatial dimension by half.
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),#Converts the 3D feature map into a 1D vector.
    Dense(128, activation='relu'),
    Dropout(0.5),#Prevents overfitting by randomly setting 50% of the neurons to zero during each training step, forcing the model to learn more robust features.

    Dense(len(Classes), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
batch_size = 32

# Calculate steps per epoch
steps_per_epoch = len(train_data) // batch_size
validation_steps = len(val_data) // batch_size

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    data_generator(batch_size, train_data),
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=data_generator(batch_size, val_data),
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)


Epoch 1/20
717/717 ━━━━━━━━━━━━━━━━━━━━ 770s 1s/step - accuracy: 0.2501 - loss: 1.8461 - val_accuracy: 0.3537 - val_loss: 1.6697
Epoch 2/20
717/717 ━━━━━━━━━━━━━━━━━━━━ 2839s 4s/step - accuracy: 0.3246 - loss: 1.6837 - val_accuracy: 0.3879 - val_loss: 1.5987
Epoch 3/20
717/717 ━━━━━━━━━━━━━━━━━━━━ 783s 1s/step - accuracy: 0.3734 - loss: 1.5850 - val_accuracy: 0.4236 - val_loss: 1.5198
Epoch 4/20
265/717 ━━━━━━━━━━━━━━━━━━━━ 7:55 1s/step - accuracy: 0.4147 - loss: 1.5053

In [31]:
# Generate the entire test set
X_test, y_test = [], []
for img, label in testing_data:
    X_test.append(img)
    y_test.append(label)

X_test = np.array(X_test) / 255.0
y_test = np.array(y_test)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


180/180 - 37s - 203ms/step - accuracy: 0.4249 - loss: 1.5047
Test Loss: 1.5046643018722534
Test Accuracy: 0.42493903636932373
